In [1]:
import json
import pandas as pd

In [116]:
with open("data/others/db") as file:
    db = file.readlines()

db = [json.loads(line.removesuffix("\n")) for line in db]

In [118]:
df_db = pd.DataFrame.from_dict(db)
df_exploded = pd.concat([df_db, df_db["metadata"].apply(pd.Series)], axis=1)

In [119]:
def convert_row(row):
    return {elem["trait_type"]: elem["value"] for elem in row}


attributes = df_exploded["attributes"].apply(convert_row)
attributes = pd.DataFrame.from_dict(attributes.tolist())

In [120]:
df_final = pd.concat([df_exploded, attributes], axis=1)
df_final["image"] = df_final["image"].str.removeprefix("ipfs://")
df_final = df_final.loc[:, ["image", "Mouth", "Background", "Hat", "Eyes", "Clothes", "Fur", "Earring"]]

In [121]:
df_final.to_csv("data/others/apes_info.csv", index=False)